In [1]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = 'plotly'
TEMPLATE = 'seaborn'

from sklearn.linear_model import LinearRegression



## Scikit-learn and Pipelines


### Agenda

- The modeling process.
- Transformers in `sklearn`.
- Pipelines.


## The modeling process

### The modeling process

<center><img src="imgs/image_0.png" width="60%"></center>

1. Create (engineer) features to best reflect the "meaning" behind data.

2. Choose a model that is appropriate to capture the relationships between features ($X$) and the target/response ($y$).

3. Select a loss function and fit the model (i.e., determine $\theta$).

4. Evaluate the model (e.g. using RMSE or $R^2$).

**We can perform all of the above directly in `sklearn`!**

### `preprocessing` and `linear_model`s

For the **feature engineering** step of the modeling pipeline, we will use `sklearn`'s [`preprocessing`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) module.

<center><img src="imgs/feature_part.png" width="30%"></center>

For the **model creation** step of the modeling pipeline, we will use `sklearn`'s [`linear_model`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model) module, as we've already seen. `linear_model.LinearRegression` is an example of an **estimator** class.

<center><img src="imgs/model_part.png" width="36%"></center>

## Transformers in `sklearn`

### Transformer classes

- **Transformers** take in "raw" data and output "processed" data. They are used for **creating features**.

- The input to a transformer should be a multi-dimensional `numpy` array.
    - Inputs can be DataFrames, but `sklearn` only looks at the values (i.e. it calls `to_numpy()` on input DataFrames).

- The output of a transformer is a `numpy` array (never a DataFrame or Series).

- Transformers, like most relevant features of `sklearn`, are **classes**, not functions, meaning you need to instantiate them and call their methods.

### Case study: Restaurant tips 🧑‍🍳

We'll continue working with our trusty `tips` dataset.

In [2]:
tips = px.data.tips()
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


### Example transformer: `Binarizer`

The `Binarizer` transformer allows us to map a quantitative sequence to a sequence of 1s and 0s, depending on whether values are above or below a threshold.

|Property|Example|Description|
|---|---|---|
|Initialize with parameters| `binar = Binarizer(thresh)` | set x=1 if x > thresh, else 0|
|Transform data in a dataset | `feat = binar.transform(data)` | Binarize all columns in `data`|

First, we need to import the relevant class from `sklearn.preprocessing`. (Tip: import just the relevant classes you need from `sklearn`.)

In [3]:
from sklearn.preprocessing import Binarizer

Let's try binarizing `'total_bill'`. We'll say a "large" bill is one that is **strictly** greater than \$20.

In [4]:
tips['total_bill'].head()

0    16.99
1    10.34
2    21.01
3    23.68
4    24.59
Name: total_bill, dtype: float64

First, we initialize a `Binarizer` object with the threshold we want.

In [5]:
bi = Binarizer(threshold=20)

Then, we call `bi`'s `transform` method and pass it the data we'd like to transform. Note that its input and output are both 2D.

In [6]:
transformed_bills = bi.transform(tips[['total_bill']]) # Must give transform a 2D array/DataFrame.
transformed_bills[:5]

/Users/aileen/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Binarizer was fitted without feature names
  warnings.warn(


array([[0.],
       [0.],
       [1.],
       [1.],
       [1.]])

### Example transformer: `StdScaler`

- `StdScaler` **standardizes** data using the mean and standard deviation of the data.

$$z(x_i) = \frac{x_i - \text{mean of } x}{\text{SD of } x}$$

- Unlike `Binarizer`, `StdScaler` **requires some knowledge (mean and SD) of the dataset before transforming**.

- As such, we need to **`fit`** an `StdScaler` transformer before we can use the `transform` method.

* Typical usage: fit transformer on a sample, use that fit transformer to transform future data.


### Example transformer: `StdScaler`

It only makes sense to standardize the already-quantitative features of `tips`, so let's select just those.

In [11]:
tips_quant = tips[['total_bill', 'size']]
tips_quant.head()

,total_bill,size
0,16.99,2
1,10.34,3
2,21.01,3
3,23.68,2
4,24.59,4


Let's initialize a `StandardScaler` object.

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
stdscaler = StandardScaler()

Note that the following **does not work!** The error message is very helpful.

In [10]:
stdscaler.transform(tips_quant)

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

Instead, we need to first call the `fit` method on `stdscaler`.

In [14]:
# This is like saying "determine the mean and SD of each column in tips_quant".
stdscaler.fit(tips_quant)

StandardScaler()

Now, `transform` will work.

In [15]:
# First column is 'total_bill', second column is 'size'.
tips_quant_z = stdscaler.transform(tips_quant)
tips_quant_z[:5]

array([[-0.31471131, -0.60019263],
       [-1.06323531,  0.45338292],
       [ 0.1377799 ,  0.45338292],
       [ 0.4383151 , -0.60019263],
       [ 0.5407447 ,  1.50695847]])

We can also access the mean and variance `stdscaler` computed for each column:

In [16]:
stdscaler.mean_

array([19.78594262,  2.56967213])

In [17]:
stdscaler.var_

array([78.92813149,  0.9008835 ])

Note that we can call `transform` on DataFrames other than `tips_quant`. We will do this often – fit a transformer on one dataset (training data) and use it to transform other datasets (test data).

In [18]:
stdscaler.transform(tips_quant.sample(5))

array([[-0.59273451, -0.60019263],
       [ 0.5047255 , -0.60019263],
       [-0.0153017 , -0.60019263],
       [-0.0333113 , -0.60019263],
       [-0.1841417 ,  0.45338292]])

### `StdScaler` summary

|Property|Example|Description|
|---|---|---|
|Initialize with parameters| `stdscaler = StandardScaler()` | z-score the data (no parameters) |
|Fit the transformer| `stdscaler.fit(X)` | Compute the mean and SD of `X`|
|Transform data in a dataset | `feat = stdscaler.transform(X_new)` | z-score `X_new` with mean and SD of `X`|

### Example transformer: `OneHotEncoder`

Let's keep just the categorical columns in `tips`.

In [19]:
tips_cat = tips[['sex', 'smoker', 'day', 'time']]
tips_cat.head()

,sex,smoker,day,time
0,Female,No,Sun,Dinner
1,Male,No,Sun,Dinner
2,Male,No,Sun,Dinner
3,Male,No,Sun,Dinner
4,Female,No,Sun,Dinner


Like `StdScaler`, we will need to `fit` our `OneHotEncoder` transformer before it can transform anything.

In [20]:
from sklearn.preprocessing import OneHotEncoder

In [21]:
ohe = OneHotEncoder()
ohe.fit(tips_cat)

OneHotEncoder()

We can look at the unique values (i.e. categories) in each column by using the `categories_` attribute:

In [23]:
ohe.categories_

[array(['Female', 'Male'], dtype=object),
 array(['No', 'Yes'], dtype=object),
 array(['Fri', 'Sat', 'Sun', 'Thur'], dtype=object),
 array(['Dinner', 'Lunch'], dtype=object)]

In [24]:
ohe.transform(tips_cat)

<244x10 sparse matrix of type '<class 'numpy.float64'>'
	with 976 stored elements in Compressed Sparse Row format>

Since the resulting matrix is **sparse** – most of its elements are 0 – `sklearn` uses a more efficient representation than a regular `numpy` array. That's no issue, though:

In [25]:
ohe.transform(tips_cat).toarray()

array([[1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       [1., 0., 1., ..., 1., 1., 0.]])

Notice that the column names from `tips_cat` are no longer stored anywhere (remember, `fit` converts the input to a `numpy` array before proceeding).

We can use the `get_feature_names_out` method on `ohe` to access the names of the one-hot-encoded columns, though:

In [54]:
ohe.get_feature_names_out() # x0, x1, x2, and x3 correspond to column names in tips_cat.

array(['sex_Female', 'sex_Male', 'smoker_No', 'smoker_Yes', 'day_Fri',
       'day_Sat', 'day_Sun', 'day_Thur', 'time_Dinner', 'time_Lunch'],
      dtype=object)

In [53]:
pd.DataFrame(ohe.transform(tips_cat).toarray(), 
             columns=ohe.get_feature_names_out()) # If we need a DataFrame back, for some reason.

,sex_Female,sex_Male,smoker_No,smoker_Yes,day_Fri,day_Sat,day_Sun,day_Thur,time_Dinner,time_Lunch
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
239,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
240,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
241,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
242,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


## Pipelines

<center><img src="imgs/image_0.png" width="50%"></center>

<br>

So far, we've used transformers for feature engineering and models for prediction. We can combine these steps into a single `Pipeline`.

### `Pipeline`s in `sklearn`

- To instantiate a `Pipeline`, we must provide a list with zero or more transformers followed by a single model.
    - All "steps" must have `fit` methods, and all but the last must have `transform` methods.
    - Template: `pl = Pipeline([feat_trans1, feat_trans2, ..., mdl])`.
    
```py

```

- Once a `Pipeline` is instantiated, you can fit **all** steps (transformers and model) using a single call to the `fit` method.
```py
pl.fit(X, y)
```

- To make predictions using **raw, untransformed data**, use `pl.predict`.

- The actual list we provide `Pipeline` with must be a list of **tuples**, where
    - The first element is a "name" (that we choose) for the step.
    - The second element is a transformer or estimator instance.

### Our first `Pipeline`

Let's build a `Pipeline` that:
- One hot encodes the categorical features in `tips`.
- Fits a regression model on the one hot encoded data.

In [28]:
tips_cat = tips[['sex', 'smoker', 'day', 'time']]
tips_cat.head()

,sex,smoker,day,time
0,Female,No,Sun,Dinner
1,Male,No,Sun,Dinner
2,Male,No,Sun,Dinner
3,Male,No,Sun,Dinner
4,Female,No,Sun,Dinner


In [29]:
from sklearn.pipeline import Pipeline

In [30]:
pl = Pipeline([
    ('one-hot', OneHotEncoder()),
    ('lin-reg', LinearRegression())
])

Now that `pl` is instantiated, we `fit` it the same way we would fit the individual steps.

In [31]:
pl.fit(tips_cat, tips['tip'])

Pipeline(steps=[('one-hot', OneHotEncoder()), ('lin-reg', LinearRegression())])

Now, to make predictions using **raw data**, all we need to do is use `pl.predict`:

In [32]:
pl.predict([['Female', 'Yes', 'Sat', 'Lunch']])

/Users/aileen/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([2.41792163])

In [33]:
pl.predict(tips_cat.iloc[:5])

array([3.10415414, 3.27436302, 3.27436302, 3.27436302, 3.10415414])

`pl` performs **both** feature transformation and prediction with just a single call to `predict`!

We can access individual "steps" of a `Pipeline` through the `named_steps` attribute:

In [34]:
pl.named_steps

{'one-hot': OneHotEncoder(), 'lin-reg': LinearRegression()}

In [35]:
pl.named_steps['one-hot'].transform(tips_cat).toarray()

array([[1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       [1., 0., 1., ..., 1., 1., 0.]])

In [36]:
pl.named_steps['lin-reg'].coef_

array([-0.08510444,  0.08510444, -0.04216238,  0.04216238, -0.20256076,
       -0.12962763,  0.13756057,  0.19462781,  0.25168453, -0.25168453])

`pl` also has a `score` method, the same way a fit `LinearRegression` instance does:

In [37]:
pl.score(tips_cat, tips['tip'])

0.027496790201475663

### More sophisticated `Pipeline`s

- In the previous example, we one hot encoded every input column. **What if we want to perform different transformations on different columns?**

- **Solution:** Use a `ColumnTransformer`.
    - Instantiate a `ColumnTransformer` using a list of tuples, where:
        - The first element is a "name" we choose for the transformer.
        - The second element is a transformer instance (e.g. `OneHotEncoder()`).
        - The third element is a **list of relevant column names**.

- `ColumnTransformer` is extremely useful, but it was only added to `sklearn` in 2018!

### Planning our first `ColumnTransformer`

In [38]:
from sklearn.compose import ColumnTransformer

Let's perform different transformations on the quantitative and categorical features of `tips` (note that we are not transforming `'tip'`).

In [39]:
tips_features = tips.drop('tip', axis=1)
tips_features.head()

,total_bill,sex,smoker,day,time,size
0,16.99,Female,No,Sun,Dinner,2
1,10.34,Male,No,Sun,Dinner,3
2,21.01,Male,No,Sun,Dinner,3
3,23.68,Male,No,Sun,Dinner,2
4,24.59,Female,No,Sun,Dinner,4


- We will leave the `'total_bill'` column untouched.

- To the `'size'` column, we will apply the `Binarizer` transformer with a threshold of 2 (big tables vs. small tables).

- To the categorical columns, we will apply the `OneHotEncoder` transformer.

- In essence, we will create a transformer that reproduces the following DataFrame:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>size</th>
      <th>x0_Female</th>
      <th>x0_Male</th>
      <th>x1_No</th>
      <th>x1_Yes</th>
      <th>x2_Fri</th>
      <th>x2_Sat</th>
      <th>x2_Sun</th>
      <th>x2_Thur</th>
      <th>x3_Dinner</th>
      <th>x3_Lunch</th>
      <th>total_bill</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>16.99</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>10.34</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>21.01</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>23.68</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>24.59</td>
    </tr>
  </tbody>
</table>


### Building a `Pipeline` using a `ColumnTransformer`

Let's start by creating our `ColumnTransformer`.

In [40]:
preproc = ColumnTransformer(
    transformers=[
        ('size', Binarizer(threshold=2), ['size']),
        ('categorical_cols', OneHotEncoder(), ['sex', 'smoker', 'day', 'time'])
    ],
    remainder='passthrough' # Specify what to do with all other columns ('total_bill' here) – drop or passthrough.
)

Now, let's create a `Pipeline` using `preproc` as a transformer, and `fit` it:

In [41]:
pl = Pipeline([
    ('preprocessor', preproc), 
    ('lin-reg', LinearRegression())
])

In [42]:
pl.fit(tips_features, tips['tip'])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('size',
                                                  Binarizer(threshold=2),
                                                  ['size']),
                                                 ('categorical_cols',
                                                  OneHotEncoder(),
                                                  ['sex', 'smoker', 'day',
                                                   'time'])])),
                ('lin-reg', LinearRegression())])

Prediction is as easy as calling `predict`:

In [43]:
tips_features.head()

,total_bill,sex,smoker,day,time,size
0,16.99,Female,No,Sun,Dinner,2
1,10.34,Male,No,Sun,Dinner,3
2,21.01,Male,No,Sun,Dinner,3
3,23.68,Male,No,Sun,Dinner,2
4,24.59,Female,No,Sun,Dinner,4


In [44]:
# Note that we fit the Pipeline using tips_features, not tips_features.head()!
pl.predict(tips_features.head())

array([2.73813307, 2.32343202, 3.3700388 , 3.36798392, 3.74755924])

We can even call each transformer in `pl['preprocessor']` individually to re-create the transformed DataFrame. (There's no _practical_ reason to do this, it's more for illustration.)

In [55]:
dfs = []
for trans in pl['preprocessor'].transformers_:
    if isinstance(trans[1], str) and trans[1] == 'passthrough':
        df = tips_features.iloc[:, trans[2]]
    else:
        vals = trans[1].transform(tips_features[trans[2]])
        columns = trans[2]
        if str(trans[1]) == 'OneHotEncoder()':
            vals = vals.toarray()
            columns = trans[1].get_feature_names_out()
        df = pd.DataFrame(vals, columns=columns)
    dfs.append(df)

pd.concat(dfs, axis=1)

,size,sex_Female,sex_Male,smoker_No,smoker_Yes,day_Fri,day_Sat,day_Sun,day_Thur,time_Dinner,time_Lunch,total_bill
0,0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,16.99
1,1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,10.34
2,1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,21.01
3,0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,23.68
4,1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,24.59
...,...,...,...,...,...,...,...,...,...,...,...,...
239,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,29.03
240,0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,27.18
241,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,22.67
242,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,17.82


### Aside: `FunctionTransformer`

A transformer you'll often use as part of a `ColumnTransformer` is the `FunctionTransformer`, which enables you to use your own functions on entire columns. Think of it as the `sklearn` equivalent of `apply`.

In [46]:
from sklearn.preprocessing import FunctionTransformer

In [47]:
f = FunctionTransformer(np.sqrt)
f.transform([1, 2, 3])

array([1.        , 1.41421356, 1.73205081])

### Aside: `joblib`

In order to make your work reproducible, you might also want to save your pipeline for future uses. You can dump your pipeline via `pickle` or `joblib`.

#### So, should I use pickle or joblib to save and load my machine learning models?

joblib is usually significantly faster on large NumPy arrays because it has special implementations for them. If your model contains large NumPy arrays (as the vast majority of models do), then joblib should be faster than pickle.

If you don’t need to save NumPy arrays, then pickle may be significantly faster, especially on large collections of native Python objects because the pickle module is implemented in C while joblib is pure Python

In [59]:
import joblib

#To save your pipeline

joblib.dump(pl,'pipe.joblib')

['pipe.joblib']

In [60]:
#To load your pipeline from file

same_pipe = joblib.load('pipe.joblib')

same_pipe.predict(tips_features.head())


array([2.73813307, 2.32343202, 3.3700388 , 3.36798392, 3.74755924])

In [61]:
pl.predict(tips_features.head())

array([2.73813307, 2.32343202, 3.3700388 , 3.36798392, 3.74755924])

#### Keeping track of all the pipelines.

In this notebook (and in life) we will want to keep track of all our models.  Here I will store the models in a dictionary with a (not great) name so I can remember which model is which and can easily compare my models in a plot.

In [62]:
models = {"model1": pl}

### Summary: `Pipeline`s

- `Pipeline`s are powerful because they allow you to perform feature engineering and training/prediction all through a single object.

- It's important to understand what each step of a `Pipeline` does. Neural networks work _similarly_ to `sklearn` `Pipeline`s, in that they follow a well-defined sequence of steps to make predictions.

- Pipelines in `sklearn` combine one or more transformers with a single model (estimator), allowing us to perform feature engineering and prediction through a single object.